Install Required Libraries

In [13]:
!pip install ultralytics torch torchvision opencv-python pillow


Initialize the YOLOv8 Object Detection Model

In [14]:
from ultralytics import YOLO

# Load the YOLOv8 model (ensure that you have the 'yolov8s.pt' model file)
yolo_model = YOLO('yolov8s.pt')


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Asus TUF Gaming F15\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

Load Age and Gender Detection Models

In [ ]:
import torch

# Load AgeNet and GenderNet models
age_model = torch.load('agenet.pth')  # Ensure the correct path to the model file
age_model.eval()  # Set the model to evaluation mode

gender_model = torch.load('gendernet.pth')  # Ensure the correct path to the model file
gender_model.eval()  # Set the model to evaluation mode


Define Preprocessing for Age and Gender Models

In [ ]:
from torchvision import transforms

# Define preprocessing transformation for face images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])


Helper Functions for Age and Gender Prediction

In [ ]:
from PIL import Image

# Function to predict age
def predict_age(face_image):
    face_tensor = transform(face_image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        age_prediction = age_model(face_tensor)
    age = age_prediction.argmax().item()  # Assuming age model gives classification output
    return age

# Function to predict gender
def predict_gender(face_image):
    face_tensor = transform(face_image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        gender_prediction = gender_model(face_tensor)
    gender = "Male" if gender_prediction.argmax().item() == 1 else "Female"
    return gender


Capture Video Input

In [ ]:
import cv2

# Open video capture (0 for default camera)
cap = cv2.VideoCapture(0)

# Test if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera")
    exit()  # Exit if the camera can't be opened


Perform Object Detection, Age and Gender Detection

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform object detection using YOLOv8
    results = yolo_model(frame)  # YOLOv8 performs detection
    
    # Loop through the detected objects (bounding boxes, labels, etc.)
    for result in results.xyxy[0]:  # Loop through each detection in the frame
        x1, y1, x2, y2, confidence, label = result  # Extract bounding box coordinates and label
        label = int(label.item())  # Convert tensor label to integer
        
        # Assuming label '0' corresponds to 'face'
        if label == 0:  # Adjust label if needed for face detection
            # Extract the face from the frame
            face = frame[int(y1):int(y2), int(x1):int(x2)]
            face_image = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))  # Convert to PIL Image

            # Predict age and gender
            age = predict_age(face_image)
            gender = predict_gender(face_image)

            # Draw bounding box and annotate with age and gender
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f"Age: {age}", (int(x1), int(y1) - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.putText(frame, f"Gender: {gender}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the video with annotations
    cv2.imshow('Facial Recognition System', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


Clean Up

In [ ]:
# Release the video capture and destroy any OpenCV windows
cap.release()
cv2.destroyAllWindows()
